# Stock Reports

In [1]:
import os
import sys
import pickle
from time import sleep
from datetime import datetime as dt, timedelta

from config import *
from alpaca_functions import *

import numpy as np
import pandas as pd
import math
from itertools import chain

from bs4 import BeautifulSoup
import requests
import datetime
from dateutil import parser
from requests.models import DecodeError

import yfinance
import yahoo_fin.stock_info as si
#from talib import RSI, BBANDS

import alpaca_trade_api as tradeapi

#import google-api
from trade import *

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL)

## Helper Functions

In [3]:
def buying_power():

    account = api.get_account()
    
    if account.trading_blocked:
        print('Account is currently restricted from trading.')
    
    buying_power = account.buying_power
    print('${} is available as buying power.'.format(buying_power))
    
    return(buying_power)

def stockQuantity(ticker, cashCap = 10000):
    
    ticker_yahoo = yfinance.Ticker(ticker)
    data = ticker_yahoo.history()
    last_quote = (data.tail(1)['Close'].iloc[0])
    print(ticker,last_quote)
    
    stockQty = cashCap / last_quote
    stockQty = math.floor(stockQty)
    return stockQty

def stopLoss(ticker):
    ticker_yahoo = yfinance.Ticker(ticker)
    data = ticker_yahoo.history()
    last_quote = (data.tail(1)['Close'].iloc[0])
    stop = math.ceil(last_quote - (last_quote * .1))
    return stop

def scrape_earnings_date(ticker):
    earnings_date = si.get_next_earnings_date(ticker.lower()).strptime("2013-1-25", '%Y-%m-%d').strftime('%m/%d/%y')
    return(earnings_date)

def macro_rsi(ticker):
    try:
        #stockData = yfinance.download(ticker,start,end)
        yfinanceTicker = yfinance.Ticker(ticker)
        stockData = yfinanceTicker.history(interval='1d', period="6mo")
        rsi = talib.RSI(stockData["Close"])
        currentRSI = rsi[-1]
        return(currentRSI)
    
    except ValueError or DecodeError:
        print('cannot compute macroRSI for: ' + str(ticker))

def micro_rsi(ticker):
    try:
        ticker_yahoo2 = yfinance.Ticker(ticker)
        data2 = ticker_yahoo2.history(interval='5m', period='day')
        rsi2 = talib.RSI(data2["Close"])
        currentRSI2 = rsi2[-1]
        return(currentRSI2)
    
    except ValueError or DecodeError:
        print('cannot compute microRSI for: ' + str(ticker))

def sendEmail(from_address, to_address, subject, msg):

    message = MIMEMultipart()
    
    message['Subject'] = subject
    message['From'] = from_address
    message['To'] = ", ".join(recipients)
    
    content = MIMEText(msg)
    message.attach(content)
    
    mail = smtplib.SMTP('smtp.gmail.com', 587)
    mail.ehlo()
    mail.starttls()
    mail.login(from_address, EMAIL_PW)
    mail.sendmail(from_address,to_address, message.as_string())
    
    mail.close()
        
def report_email():
    Recipient = "rsipython7898@gmail.com"
    Sender = "rsipython7898@gmail.com"
    sToken = "" #password
    port = 465
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', port, context=context) as server:

            server.login(Sender, sToken)

            Subject = "Stock Report"

            html = oversoldtickers

            msg = f'Subject: {Subject}\n\n{html}'

            server.sendmail(Sender, Recipient, msg)

def sell_email():
    Recipient = "rsipython7898@gmail.com"
    Sender = "rsipython7898@gmail.com"
    sToken = "" #password
    port = 465
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', port, context=context) as server:

            server.login(Sender, sToken)

            Subject = "Sell Order Placed"

            html = sellORDER

            msg = f'Subject: {Subject}\n\n{html}'

            server.sendmail(Sender, Recipient, msg)

def buy_email():
    Recipient = "rsipython7898@gmail.com"
    Sender = "rsipython7898@gmail.com"
    sToken = "" #password
    port = 465
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', port, context=context) as server:

            server.login(Sender, sToken)

            Subject = "Buy Order Placed"

            html = buyORDER

            msg = f'Subject: {Subject}\n\n{html}'

            server.sendmail(Sender, Recipient, msg)   

def selllog(ticker):
    sellLOG = open('sellLOG.txt', 'w')
    sellLOG.write('\nSELL ORDER PLACED')
    sellLOG.write('\n' + str(ticker))
    sellLOG.write('\n' + str(quantity) + ' to be sold ')
    sellLOG.write('\nMicroRSI: ' + str(macroRSI))
    sellLOG.write('\n============================================')

def buylog(ticker):
    buylog = open('buylog.txt', 'w')
    buylog.write('\n' + ticker)
    buylog.write('\nMicroRSI: ' + str(macroRSI))
    buylog.write('\n' + str(stockquantity) + ' stocks are availble for purchase within our costcap')
    buylog.write('\n' + str(stopPrice) + ' is the stop loss price')
    buylog.write('\n============================================')

## Stocks List

Updating current list of stocks if new positions are gained through the main website or new stocks are added into the watchlist through the main website. 

Storing the updated list in pickle and txt files for practice and readability.

In [23]:
# I want to work with pickle file for learning and txt file for readability
stocks = []


# Grab stocks from watchlist in case new stocks were added there
watchlist = api.get_watchlist('10c71436-8e6b-4d08-9b43-030cbe3ae6c2').assets

for i in range(0, len(watchlist)):
    symbol = watchlist[i]['symbol']
    stocks.append(symbol)
    
    
# Grab current positions in case new stocks were bought
current_positions = api.list_positions()

for i in range(0, len(current_positions) - 1):
    symbol = vars(current_positions[i])['_raw']['symbol']
    stocks.append(symbol)
    
    
# Grab initial stocks from given list if it exists
try:
    [stocks.append(line.rstrip()) for line in open("stocks.txt", "r")]
except (OSError, IOError) as e:
    with open('stocks.txt', 'w') as f:
        for stock in stocks:
            f.write("%s\n" % stocks) 
    [stocks.append(line.rstrip()) for line in open("stocks.txt", "r")]
        
        
# Grab stocks from updated pickle file if it exists
try:
    pickle_stocks = pickle.load(open("stocks_list.pickle", "rb"))
except (OSError, IOError) as e:
    pickle.dump(stocks, open("stocks_list.pickle", "wb"))
    pickle_stocks = pickle.load(open("stocks_list.pickle", "rb"))    

    
stocks = list(set(stocks + pickle_stocks))
stocks.sort()
    
# Store in pickle file and txt file
with open('stocks_list', 'wb') as fp:
    pickle.dump(stocks, fp)

with open('stocks.txt', 'w') as f:
    for stock in stocks:
        f.write("%s\n" % stock)

['AAP', 'AAPL', 'ABNB', 'ACN', 'ADBE', 'ADSK', 'ALGN', 'AMGN', 'AMZN', 'ANET', 'ANTM', 'ARKK', 'AVGO', 'AXP', 'AZO', 'BA', 'BABA', 'BBY', 'BDX', 'BIDU', 'BIIB', 'BKNG', 'BLK', 'BP', 'BURL', 'CAT', 'CHKP', 'CHTR', 'CI', 'CLX', 'CMG', 'COO', 'COST', 'COUP', 'CRM', 'CRWD', 'CTXS', 'CVNA', 'CYBR', 'DE', 'DG', 'DIA', 'DIS', 'DNA', 'DOCU', 'DPZ', 'ENPH', 'EQIX', 'ETSY', 'EXPE', 'FB', 'FDX', 'FFIV', 'FIS', 'FSLY', 'FVRR', 'GOOG', 'GS', 'HD', 'HELE', 'HON', 'HSY', 'HUM', 'IBM', 'INTU', 'ISRG', 'IWM', 'JAZZ', 'JBL', 'KLAC', 'LMT', 'LOW', 'LRCX', 'LULU', 'MA', 'MCD', 'MCO', 'MDB', 'MELI', 'MLM', 'MMM', 'MOH', 'MRNA', 'MSFT', 'MSTR', 'MTCH', 'MTUM', 'NCLH', 'NFLX', 'NKE', 'NOW', 'NSC', 'NVDA', 'NXPI', 'OKTA', 'ORLY', 'PANW', 'PAYC', 'PCTY', 'PG', 'PNC', 'PPG', 'PYPL', 'QCOM', 'QDEL', 'QQQ', 'QRVO', 'RACE', 'RBLX', 'RCL', 'REGN', 'RGEN', 'RNG', 'ROKU', 'ROST', 'RUTH', 'SAM', 'SEDG', 'SHOP', 'SHW', 'SNOW', 'SOXL', 'SPGI', 'SPLK', 'SPOT', 'SPY', 'SQ', 'STE', 'STMP', 'STZ', 'SWK', 'SWKS', 'TDOC', 'TE

In [24]:
portfolio_allocation()

99.52988247061766

In [ ]:
tickers = []
results_list = []
oversoldtickers = ''

today = dt.now()
start = today - timedelta(days=365)
hour = int(today.strftime("%H"))
current_time = today.strftime("%H:%M")

print("Stock Report begun at ", current_time)

pa = portfolio_allocation()

#logfile = open('Morning_Report.txt', 'w')
#logfile.write('Morning Report ' + str(date))
#logfile.write('\n\n')
#logfile.write('Portfolio Allocation: ' + str(pa))
#logfile.close

for ticker in stocks:
        try:
            positions = get_positions(ticker)
            if positions>=1:
                logfile.write('\nPosition found for ' + str(ticker))
                currentPositions.append(ticker)
        except:
            logfile.write('\ncannot find current position for ' + str(ticker))

for ticker in stocks:
    earnings = scrape_earnings_date(ticker)
    try:
        earningConv = parser.parse(earnings)
        earningsCheck = earningConv - date
        macroRSI = macro_rsi(ticker)
        if macroRSI <= 35:
            macroRSI = macro_rsi(ticker)
            logfile.write('\n' + ticker)
            logfile.write('\nOversold')
            logfile.write('\nRSI ' + str(macroRSI))
            logfile.write('\nDays until earnings: ' + str(earningsCheck.days))
            logfile.write('\n=====================================')
            tickers.append(ticker)
            oversoldticker = '\n\n' + str(ticker) + '\n' + 'OVERSOLD' + '\n' + 'RSI: ' + str(macroRSI) + '\nDays until earnings: ' + str(earningsCheck.days)
            oversoldtickers = oversoldtickers + oversoldticker
    except:
        logfile.write('\n' + ticker)
        logfile.write('\nUnable to gather data')
        logfile.write('\n=====================================')

report_email()

logfile.write('\n\n Trading Starting' + str(date))

while hour < 16:

    sleep(120)
    end = datetime.datetime.now()
    pa = portfolio_allocation()
    logfile.write('\n\nPortfolio Allocation: ' + str(pa))

    for ticker in currentPositions:
            up, mid, low = BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
            macroRSI = macro_rsi(ticker)
            print(ticker)
            print(str(macroRSI))
            try:
                if macroRSI >= 60:
                    side = 'sell'
                    type = 'market'
                    quantity = get_positions(ticker)
                    time_in_force = 'gtc'
                    selllog(ticker)
                    currentPositions.remove(ticker)
                    create_order(ticker, quantity, side, type, time_in_force)
                    sellORDER = '\n\n' + str(ticker) + '\n' + 'SELL ORDER PLACED' + '\n' + 'RSI: ' + str(macroRSI) 
                    currentPositions.remove(ticker)
                    tickers.append(ticker)
                    selllog(ticker)
                    sell_email()
                
            except:
                number = 1

    while pa > 30 and hour < 16:
        sleep(140)
        for ticker in currentPositions:
            macroRSI = macro_rsi(ticker)
            print(ticker)
            print(str(macroRSI))
            try:
                if macroRSI >= 60:
                    side = 'sell'
                    type = 'market'
                    quantity = get_positions(ticker)
                    time_in_force = 'gtc'
                    currentPositions.remove(ticker)
                    create_order(ticker, quantity, side, type, time_in_force)
                    sellORDER = '\n\n' + str(ticker) + '\n' + 'SELL ORDER PLACED' + '\n' + 'RSI: ' + str(macroRSI)
                    currentPositions.remove(ticker)
                    tickers.append(ticker)
                    selllog(ticker)
                    sell_email()
            except:
                number = 1
        for ticker in currentPositions:
            try:
                positions = get_positions(ticker)
                if positions >= 1:
                    logfile.write('\nPre-existing position in portfolio for ' + str(ticker))
                    tickers.remove(ticker)
            except:
                logfile.write('\ncannot find current position for ' + ticker)
        for ticker in tickers:
            pa = portfolio_allocation()
            if pa < 30:
                break
            try:
                macroRSI = macro_rsi(ticker)
                print(ticker)
                print(str(macroRSI))
            except:
                logfile.write('\ncould not calculate rsi for ' + ticker)
            try:
                if macroRSI <= 35:
                    stockquantity = stockQuantity(ticker)
                    stopPrice = stopLoss(ticker)
                    side = 'buy'
                    type = 'market'
                    quantity = str(stockquantity)
                    time_in_force = 'gtc'
                    buylog(ticker)
                    create_order(ticker, quantity, side, type, time_in_force)
                    buyORDER = '\n\n' + str(ticker) + '\n' + 'BUY ORDER PLACED' + '\n' + 'RSI: ' + str(macroRSI)
                    currentPositions.append(ticker)
                elif macroRSI >= 36:
                    logfile.write('\n' + str(ticker))
                    logfile.write('\nMacroRSI: ' + str(macroRSI))
                else:
                    number = 1   
            except:
                logfile.write('\ncannot gather data')
    hour = datetime.datetime.now()
    hour = hour.time()
    hour = int(hour.strftime("%H"))
hour = datetime.datetime.now()
hour = hour.time()
hour = int(hour.strftime("%H"))   